In [1]:
import pandas as pd
import numpy as np
import os
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle 
from numpy import loadtxt
from keras.models import load_model
pd.set_option('display.max_columns', None)

In [2]:
big_df = pd.read_csv("data/big_df.csv")
big_df

,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_weight,red_weight,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner
0,0,35.0,29.0,66.0,71.0,TJ Dillashaw,Cory Sandhagen,67.0,70.0,Orthodox,Switch,1.0,0.5,1341.0,476.0,2832.0,906.0,1005.0,386.0,2438.0,800.0,21.0,4.0,56.0,8.0,12.0,2.0,2751.0,671.0,9.0,4.0,0.0,2.0,140.0,65.0,223.0,98.0,657.0,186.0,1980.0,508.0,150.0,33.0,191.0,46.0,665.0,203.0,2005.0,512.0,104.0,14.0,157.0,

In [3]:
# pred_df = pd.read_csv("data/rf_preds.csv")
# pred_df

In [4]:
# pred_df = pred_df.sort_values(by="date")
# mcblue = pred_df[pred_df["blue_name"] == "Conor McGregor"]
# mcred = pred_df[pred_df["red_name"] == "Conor McGregor"]
# mcg = pd.concat([mcblue, mcred])
# mcg

In [5]:
future_df = big_df[big_df["winner"] == "tbd"]
future_df

,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_weight,red_weight,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner
0,0,35.0,29.0,66.0,71.0,TJ Dillashaw,Cory Sandhagen,67.0,70.0,Orthodox,Switch,1.0,0.5,1341.0,476.0,2832.0,906.0,1005.0,386.0,2438.0,800.0,21.0,4.0,56.0,8.0,12.0,2.0,2751.0,671.0,9.0,4.0,0.0,2.0,140.0,65.0,223.0,98.0,657.0,186.0,1980.0,508.0,150.0,33.0,191.0,46.0,665.0,203.0,2005.0,512.0,104.0,14.0,157.0,

In [6]:
number = LabelEncoder()
# data['red_name'] = number.fit_transform(data['red_name'].astype('str'))
# data['blue_name'] = number.fit_transform(data['blue_name'].astype('str'))
future_df['red_stance'] = number.fit_transform(future_df['red_stance'].astype('str'))
future_df['blue_stance'] = number.fit_transform(future_df['blue_stance'].astype('str'))
future_df

<ipython-input-6-be9019f59664>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df['red_stance'] = number.fit_transform(future_df['red_stance'].astype('str'))
<ipython-input-6-be9019f59664>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df['blue_stance'] = number.fit_transform(future_df['blue_stance'].astype('str'))


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_weight,red_weight,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner
0,0,35.0,29.0,66.0,71.0,TJ Dillashaw,Cory Sandhagen,67.0,70.0,0,2,1.0,0.5,1341.0,476.0,2832.0,906.0,1005.0,386.0,2438.0,800.0,21.0,4.0,56.0,8.0,12.0,2.0,2751.0,671.0,9.0,4.0,0.0,2.0,140.0,65.0,223.0,98.0,657.0,186.0,1980.0,508.0,150.0,33.0,191.0,46.0,665.0,203.0,2005.0,512.0,104.0,14.0,157.0,26.0,178.0,6

In [7]:
data = future_df.drop(columns=["lev1", "winner", \
                                   "red_name", "blue_name"
                                  ])
target = future_df["winner"]
# feature_names = data.columns

In [11]:
loaded_model = load_model("saved_models/neural_model.h5")

pred = loaded_model.predict(data)
pred
# future_df["prediction"] = pred
# future_df

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [ ]:
pcts = loaded_model.predict_proba(data)
favs = []
dogs = []
for pct in pcts:
    fav = max(pct)
    dog = min(pct)
    favs.append(fav)
    dogs.append(dog)

In [ ]:
future_df["fav_pct"] = favs
future_df["dog_pct"] = dogs
future_df

In [ ]:
future_df["fav_pct"] = future_df["fav_pct"].round(2)
future_df["dog_pct"] = future_df["dog_pct"].round(2)
future_df

In [ ]:
future_df["fav_moneyline"] = (future_df["fav_pct"] * 100) / (100 - (future_df["fav_pct"] * 100)) * (-100)
future_df["dog_moneyline"] = ((100 - (future_df["dog_pct"] * 100)) / (future_df["dog_pct"] * 100) * 100)
future_df

In [ ]:
future_df["fav_moneyline"] = future_df["fav_moneyline"].astype(int)
future_df["dog_moneyline"] = future_df["dog_moneyline"].astype(int)
future_df

In [ ]:
future_df.to_csv("data/neaural_future_preds.csv")

In [ ]:
my_picks = future_df[["blue_name", "red_name", "prediction", "fav_pct", "dog_pct", "fav_moneyline", "dog_moneyline"]]
my_picks.to_csv("data/neaural_picks.csv")
my_picks